# Scrape meter data from Measurabl

This is a script to get meter data in Measurabl (https://www.measurabl.com/), which is a ESG software for commercial real estate.  
Measurabl is an ESG system for commercial real estate. It helps to keep track of asset information, meter information, usage amount, cost etc. and create reports.  

We can get a list of assets at once, but not for a list of meters and a list of data points.
To get them, I need to go to a home page first, go to an individual asset page, and move on to a utility tab and click a download button of each meter. This takes quite a while for me to get all the meters for all the assets. Instead of creating the list manually, I wrote a script below to get the meter list.

In [ ]:
import urllib3
import pandas as pd
import certifi
import re
import time
import datetime
import os
from selenium import webdriver
import chromedriver_binary
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

In [ ]:
# Config
HOME_URL = 'https://app.measurabl.com/' # サインイン

# login info
USER = "******"
PW = "******"

# Meter list
df = pd.read_excel('measurabl_sites_meter.xlsx')　　 # meter master file
df.head()

# PATH
DOWNLOAD_PATH = os.path.join(os.pardir, os.pardir, 'Downloads')

TODAY_ISO = datetime.date.today().isoformat()

In [ ]:
def create_meter_list():
    # DOWNLOAD_PATH = os.path.join(os.pardir, os.pardir, 'Downloads')
    creation_times = [(f, os.path.getctime(os.path.join(DOWNLOAD_PATH, f))) for f in os.listdir(DOWNLOAD_PATH) if f.endswith(".csv")]
    creation_times.sort(key=lambda x: x[1])  # sort by creation time
    latest_file = creation_times[-1][0]
    
    df = pd.read_csv(os.path.join(DOWNLOAD_PATH, latest_file))
    df['Asset_name'] = n
    
    return df

In [ ]:
def get_meters():
    """
    Measurablにログインし、各アセットのメーターの情報を取得する
    """
    driver = webdriver.Chrome()
    driver.get(HOME_URL)
    driver.implicitly_wait(3) #画面に要素がロードされるまで3秒待つ（ページを開いた直後には指定したい要素が読み込まれないことがあるため）


    # ログインする
    elem = driver.find_element_by_id("user_email")
    elem.clear()
    elem.send_keys(USER)

    elem = driver.find_element_by_id("user_password")
    elem.clear()
    elem.send_keys(PW)

    # ログインボタンを選択して、クリックする
    elem = driver.find_element_by_xpath("//button[@type='submit'][@class=' btn btn-success']")
    elem.click()


    # オリエンテーションの画面が出るまで20秒待つ
    time.sleep(10)
    # driver.implicitly_wait(20)

    # オリエンテーションのポップアップが出るので、escapeで閉じる
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
    
    # driver = webdriver.Chrome()
    df_meter = []
    for u, n, e, f, w in zip(df['URL'], df['Name'], df['Electric'], df['Fuel'], df['Water']):
        driver.get(u)
        driver.implicitly_wait(10) #画面に要素がロードされるまで3秒待つ（ページを開いた直後には指定したい要素が読み込まれないことがあるため）
        
        # Electric
        if e > 0:
            print(n)
            time.sleep(3)
            driver.find_element_by_xpath("//*[@id='electricMeters']/div[1]/div/button").click()
            time.sleep(3)
            df_= create_meter_list()
            # df_ = pd.read_excel("")
            df_meter.append(df_)
        
        # Fuel    
        if f > 0:
            time.sleep(3)
            driver.find_element_by_xpath("//*[@id='fuelMeters']/div[1]/div/button").click()
            time.sleep(3)
            df_= create_meter_list()
            df_meter.append(df_)
            
        # District
        if d > 0:
            time.sleep(2)
            element = driver.find_element_by_xpath("//*[@id='districtMeters']/div[1]/div/button")
            driver.execute_script("arguments[0].click();", element)
            time.sleep(2)
            df_= create_meter_list()
            df_meter = df_meter.append(df_)
            
        # Water
        if w > 0:
            time.sleep(2)
            driver.find_element_by_xpath("//*[@id='contentBody']/div/div/div[2]/div/site-utilities/div/sub-nav/nav/ul/li[2]").click()
            time.sleep(2)
            driver.find_element_by_xpath("//*[@id='waterMeters']/div[1]/div/button").click()
            time.sleep(3)
            df_ = create_meter_list()
            df_meter.append(df_)

In [ ]:
    driver = webdriver.Chrome()
    driver.get(HOME_URL)
    driver.implicitly_wait(3) #画面に要素がロードされるまで3秒待つ（ページを開いた直後には指定したい要素が読み込まれないことがあるため）


    # ログインする
    elem = driver.find_element_by_id("user_email")
    elem.clear()
    elem.send_keys(USER)

    elem = driver.find_element_by_id("user_password")
    elem.clear()
    elem.send_keys(PW)

    # ログインボタンを選択して、クリックする
    elem = driver.find_element_by_xpath("//button[@type='submit'][@class=' btn btn-success']")
    elem.click()


    # オリエンテーションの画面が出るまで20秒待つ
    time.sleep(10)
    # driver.implicitly_wait(20)

    # オリエンテーションのポップアップが出るので、escapeで閉じる
    webdriver.ActionChains(driver).send_keys(Keys.ESCAPE).perform()
    
    # driver = webdriver.Chrome()
    df_meter = pd.DataFrame()
    for u, n, e, f, d, w in zip(df['URL'], df['Name'], df['Electric'], df['Fuel'], df['District'], df['Water']):
        driver.get(u)
        driver.implicitly_wait(10) #画面に要素がロードされるまで3秒待つ（ページを開いた直後には指定したい要素が読み込まれないことがあるため）
        print(n)
        # Electric
        if e > 0:
            time.sleep(2)
            driver.find_element_by_xpath("//*[@id='electricMeters']/div[1]/div/button").click()
            time.sleep(2)
            df_= create_meter_list()
            # df_ = pd.read_excel("")
            df_meter = df_meter.append(df_)
        
        # Fuel    
        if f > 0:
            time.sleep(2)
            element = driver.find_element_by_xpath("//*[@id='fuelMeters']/div[1]/div/button")
            driver.execute_script("arguments[0].click();", element)
            time.sleep(2)
            df_= create_meter_list()
            df_meter = df_meter.append(df_)
            
        # District
        if d > 0:
            time.sleep(2)
            element = driver.find_element_by_xpath("//*[@id='districtMeters']/div[1]/div/button")
            driver.execute_script("arguments[0].click();", element)
            time.sleep(2)
            df_= create_meter_list()
            df_meter = df_meter.append(df_)
            
        # Water
        if w > 0:
            time.sleep(2)
            driver.find_element_by_xpath("//*[@id='contentBody']/div/div/div[2]/div/site-utilities/div/sub-nav/nav/ul/li[2]").click()
            time.sleep(2)
            driver.find_element_by_xpath("//*[@id='waterMeters']/div[1]/div/button").click()
            time.sleep(3)
            df_ = create_meter_list()
            df_meter = df_meter.append(df_)


In [ ]:
df_meter = df_meter[['Asset_name', 'Name', 'Type', 'Serving', 'Readings', 'Last Reading Period']]
df_meter.to_csv("meter_list_{}.csv".format(TODAY_ISO), index=False)